# Linear Regression Project - SOLUTIONS

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [3]:
df = spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
df.show(2)

+---------+-----------+---+------------------+----------+------+------+-----------------+----+
|Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|    Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 2 rows



In [7]:
df.describe().show(1)

+-------+---------+-----------+---+-------+----------+------+------+-----------------+----+
|summary|Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+-------+---------+-----------+---+-------+----------+------+------+-----------------+----+
|  count|      158|        158|158|    158|       158|   158|   158|              158| 158|
+-------+---------+-----------+---+-------+----------+------+------+-----------------+----+
only showing top 1 row



Cruise_line  işletici firma ismi string. Bunu sayısal/katagorik olarak kullannmak istersek aşağıdakine benzer bir işlem yapacağız

In [7]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [9]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)
indexed.head(2)

[Row(Ship_name=u'Journey', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name=u'Quest', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)]

In [11]:
indexed.show(2)

+---------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|    Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
only showing top 2 rows



indexed tablonun tamamını yeni şeklini de dahil ederek tutuyor buna dikkat edelim

In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [14]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

Bu noktada linear regresyon bizden feature lar ve tahmine edilecek kolon olmak üzere [featurelar, tahmin kolonu] şeklinde bir format bekliyor.. bunu oluşturmak için daha önce gördüğümüz vectorAssembler kullanılacak. sadece age ten sonrasını aldığımıza dikkat edin

In [17]:
assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'cruise_cat'],
    outputCol="features")

yukarıda assembler vektörü age, tonnage...cruise_cat kolonlarını alarak output koonlardan feature ları oluşturmak için model oluyor Bu model assembler ile indexed i birlikte kullanarak transform yardımıyla sadece ilgili kolonları alıp buradan aşağıda TOPLAM output TABLOSU elde ediliyor


In [20]:
output = assembler.transform(indexed)

In [23]:
output.show(1)

+---------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
only showing top 1 row



In [25]:
output.select("features", "crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [29]:
final_data = output.select("features", "crew")

In [30]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [31]:
from pyspark.ml.regression import LinearRegression
# Linear Regression Modeli Oluştur ve crew tahmin kolonunu kullan
lr = LinearRegression(labelCol='crew')

In [32]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [18]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.0145423814068,0.0137445818936,-0.111000735058,0.422234330769,0.705574105078,-0.00631202648669,0.0306212943631] Intercept: -0.5598623529951635


In [33]:
test_results = lrModel.evaluate(test_data)

In [34]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 0.772375257082
MSE: 0.596563537753
R2: 0.942004240239


In [35]:
# R2 of 0.86 is pretty good, let's check the data a little closer
from pyspark.sql.functions import corr

In [36]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [37]:
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



Okay, so maybe it does make sense! Well that is good news for us, this is information we can bring to the company!


Hope you enjoyed your first consulting gig!
# Great Job!